In [1]:
import xarray as xr
import numpy as np

In [2]:
data = xr.open_zarr("../../Data/Rheology_emulation/2025_UG_projects/pairs_evp_1200itr_long.zarr")
data

<xarray.Dataset> Size: 12GB
Dimensions:        (feature: 10, z: 126676851, label: 2)
Coordinates:
  * feature        (feature) <U7 280B 'siconc' 'sithic' ... 'sidive' 'sishea'
  * label          (label) <U6 48B 'sivelv' 'sivelu'
    lat            (z) float32 507MB ...
    lon            (z) float32 507MB ...
    pair           (z) int64 1GB ...
    time_features  (z) object 1GB ...
    time_labels    (z) object 1GB ...
    x              (z) int64 1GB ...
    y              (z) int64 1GB ...
Dimensions without coordinates: z
Data variables:
    features       (feature, z) float32 5GB ...
    labels         (label, z) float32 1GB ...

In [3]:
data_80N = data.where(data.lat>80, drop=True)
data_80N

<xarray.Dataset> Size: 1GB
Dimensions:        (feature: 10, z: 13036676, label: 2)
Coordinates:
  * feature        (feature) <U7 280B 'siconc' 'sithic' ... 'sidive' 'sishea'
  * label          (label) <U6 48B 'sivelv' 'sivelu'
    lat            (z) float32 52MB 80.01 80.02 80.03 ... 80.27 80.16 80.05
    lon            (z) float32 52MB 160.1 160.8 161.4 162.1 ... 73.0 73.0 73.0
    pair           (z) int64 104MB 0 0 0 0 0 0 0 ... 563 563 563 563 563 563 563
    time_features  (z) object 104MB 1976-01-01 00:15:00 ... 1976-12-01 23:15:00
    time_labels    (z) object 104MB 1976-01-01 00:45:00 ... 1976-12-01 23:45:00
    x              (z) int64 104MB 373 374 375 376 377 ... 1146 1147 1148 1149
    y              (z) int64 104MB 1119 1119 1119 1119 ... 1205 1205 1205 1205
Dimensions without coordinates: z
Data variables:
    features       (feature, z) float32 521MB 0.997 0.997 ... 9.469e-07
    labels         (label, z) float32 104MB 0.0331 0.03183 ... -0.01731 -0.01857

In [4]:
# data_80N_Jan = data_80N.where(data_80N.time_features.dt.month==[12,1,2], drop=True)
# months = data_80N['time_features'].dt.month.values
# sel_idx = np.where(np.isin(months, [12, 1, 2]))[0]

# data_80N_DJF = data_80N.isel(z=sel_idx)

In [5]:
data['features'].sel(feature='siconc')

<xarray.DataArray 'features' (z: 126676851)> Size: 507MB
[126676851 values with dtype=float32]
Coordinates:
    feature        <U7 28B 'siconc'
    lat            (z) float32 507MB -78.02 -78.03 -78.04 ... 66.96 66.85 66.74
    lon            (z) float32 507MB ...
    pair           (z) int64 1GB ...
    time_features  (z) object 1GB ...
    time_labels    (z) object 1GB ...
    x              (z) int64 1GB ...
    y              (z) int64 1GB ...
Dimensions without coordinates: z

In [6]:
lat_vals = data_80N['lat'].values
d_da = xr.DataArray(lat_vals[np.newaxis, :],
                    dims=('feature', 'z'),
                    coords={'feature': ['lat'], 'z': data_80N.coords['z']})
concat_features = xr.concat([data_80N['features'], d_da], dim='feature')
concat_features.feature

<xarray.DataArray 'feature' (feature: 11)> Size: 308B
array(['siconc', 'sithic', 'sivelv', 'sivelu', 'utau_ai', 'utau_oi', 'vtau_ai',
       'vtau_oi', 'sidive', 'sishea', 'lat'], dtype='<U7')
Coordinates:
  * feature  (feature) <U7 308B 'siconc' 'sithic' 'sivelv' ... 'sishea' 'lat'

In [7]:
existing = list(data_80N['features'].coords['feature'].values)

# put the new DataArray into the Dataset (in-memory)
data_80N_Jan = data_80N.assign({'features': concat_features})

In [8]:
ds = xr.Dataset()
ds['features'] = concat_features
ds['labels'] = data_80N['labels']
ds

<xarray.Dataset> Size: 1GB
Dimensions:        (feature: 11, z: 13036676, label: 2)
Coordinates:
  * feature        (feature) <U7 308B 'siconc' 'sithic' ... 'sishea' 'lat'
  * z              (z) int64 104MB 0 1 2 3 ... 13036673 13036674 13036675
  * label          (label) <U6 48B 'sivelv' 'sivelu'
    lat            (z) float32 52MB 80.01 80.02 80.03 ... 80.27 80.16 80.05
    lon            (z) float32 52MB 160.1 160.8 161.4 162.1 ... 73.0 73.0 73.0
    pair           (z) int64 104MB 0 0 0 0 0 0 0 ... 563 563 563 563 563 563 563
    time_features  (z) object 104MB 1976-01-01 00:15:00 ... 1976-12-01 23:15:00
    time_labels    (z) object 104MB 1976-01-01 00:45:00 ... 1976-12-01 23:45:00
    x              (z) int64 104MB 373 374 375 376 377 ... 1146 1147 1148 1149
    y              (z) int64 104MB 1119 1119 1119 1119 ... 1205 1205 1205 1205
Data variables:
    features       (feature, z) float32 574MB 0.997 0.997 0.997 ... 80.16 80.05
    labels         (label, z) float32 104MB 0.0331 0.03183 ... -0.01731 -0.01857

In [9]:
# remove any stored encoding metadata that may contain a wrong compressor/class
for name, var in ds.items():
    try:
        var.encoding.clear()
    except Exception:
        ds[name].encoding = {}

for name in list(ds.coords):
    try:
        ds.coords[name].encoding.clear()
    except Exception:
        ds.coords[name].encoding = {}

# Now write (no explicit compressor — let zarr choose defaults)
ds.to_zarr("../data/raw/evp_1200itr_80N.zarr", mode="w")

/opt/miniconda3/envs/ML/lib/python3.13/site-packages/zarr/core/dtype/npy/string.py:249: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=7, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future version of Zarr Python. Arrays stored with this data type may be unreadable by other Zarr libraries. Use this data type at your own risk! Check https://github.com/zarr-developers/zarr-extensions/tree/main/data-types for the status of data type specifications for Zarr V3.
  v3_unstable_dtype_warning(self)
/opt/miniconda3/envs/ML/lib/python3.13/site-packages/zarr/core/dtype/npy/string.py:249: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=6, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future version of Zarr Python. Arrays s

In [10]:
test = xr.open_zarr("../data/raw/evp_1200itr_80N.zarr")
test.features.mean(dim='z')

<xarray.DataArray 'features' (feature: 11)> Size: 44B
array([ 9.2990613e-01,  2.3941922e+00, -2.7707606e-03, -2.1063232e-04,
        1.2316424e-03, -3.0568806e-06, -3.8541721e-03,  3.9279251e-03,
       -8.3721530e-11,  4.9494446e-07,  8.3671997e+01], dtype=float32)
Coordinates:
  * feature  (feature) <U7 308B 'siconc' 'sithic' 'sivelv' ... 'sishea' 'lat'